# <center><font color=#76B900><b>**Homberger problem with  NVIDIA cuOpt (Optional)**</b></font></center>
---

**Learning Objectives:**
- Demonstrate the ability of NVIDIA cuOpt to scale to the largest publicly available CVRPTW benchmark instances.

In [1]:
import cudf
import cuopt
from scipy.spatial import distance
import numpy as np
import pandas as pd
import helper_function.helper_data as helper_data
import helper_function.helper_map as helper_map

### **Extract Data**
We have a built-in utility in NVIDIA cuOpt that reads and processes the instance file to return vehicle capacity, number of vehicles and a Cuda DataFrame containing the order information. This utility is specific to a Gehring & Homberger instance file.

In [2]:
# Extract the data
df, vehicle_capacity, n_vehicles = helper_data.read_data('/home/cuopt_user/data/homberger_1000_customer_instances/C1_10_1.TXT')

/home/cuopt_user/miniconda/conda/envs/cuopt/lib/python3.8/site-packages/cudf/core/indexed_frame.py:2328: FutureWarning: The append method is deprecated and will be removed in a future version. Use cudf.concat instead.
  warnings.warn(


#### Let us take a look at the extracted data

In [3]:
print("Number of locations          : ", df["demand"].shape[0]-1)
print("Number of vehicles available : ", n_vehicles)
print("Capacity of each vehicle     : ", vehicle_capacity)
print("Initial Orders information")
print(df)

Number of locations          :  1000
Number of vehicles available :  250
Capacity of each vehicle     :  200
Initial Orders information
      vertex      x      y  demand  earliest_time  latest_time  service_time
0          0  250.0  250.0       0              0         1824             0
1          1  387.0  297.0      10            200          270            90
2          2    5.0  297.0      10            955         1017            90
3          3  355.0  177.0      20            194          245            90
4          4   78.0  346.0      30            355          403            90
...      ...    ...    ...     ...            ...          ...           ...
996      996  330.0  242.0      30            627          671            90
997      997  332.0  249.0      30             82          144            90
998      998  375.0   80.0      10            550          598            90
999      999   94.0  235.0      20            227          266            90
1000    1000  287

## **Let's Run NVIDIA cuOpt**

We have all our data ready, it's time to run NVIDIA cuOpt! But, before we launch NVIDIA cuOpt and get our optimized routes, let's dig a little deeper into the workflow step by step.

### **Step 1: Get the cost matrix ready**
To be able to find and optimize the routes for the vehicles, we first need to have information about the delivery locations and the distance between each location. This can either be provided as a cost matrix or a set of X and Y coordinates of all the locations.

This function converts the X and Y coordinates from the dataset into a Euclidean distance matrix. Skip this step if you already have a cost matrix ready or want to initialize your data model directly with X and Y coordinates (We'll talk more on this later)

In [4]:
# Build euclidean distance matrix from the x, y coordinates obtained from the dataset
# This later helps in mapping.
def build_matrix(df):
    coords = list(zip(df['x'].to_arrow().to_pylist(),
                      df['y'].to_arrow().to_pylist()))
    distances = distance.cdist(coords, coords, 'euclidean')
    return cudf.DataFrame(distances).astype(np.float32)

### **Step 2: Create the Data Model**
First, we need to create and initialize the data model with the number of vehicles in the fleet and number of nodes (delivery locations) to be visited.
Next, we need to provide the cost/distance information. This can be provided in two ways using set_matrix or set_coordinates API.
<br><br>

<b>set_matrix</b> takes in a square cost matrix containing the cost of travel which can be distance, time or any other metric, taken pairwise, between all locations.<details><summary>Read more</summary>
<p>Diagonal elements should be 0.
NVIDIA cuOpt supports unsymmetric matrices. However, we have restricted matrix support to only symmetric matrices for this demonstration.</p></details>
<br>

<b>set_coordinates</b> takes in X and Y coordinates for all locations (also known as position list). Valid coordinates are real numbers and euclidean distance is used to measure the distance between two locations.
<br><br>

We can additionally set the vehicle priorities if desired using <b>set_vehicle_priorities</b> where 0 is the highest priority and INT_MAX is the lowest priority. <details><summary>Read more</summary> <p>Higher priority vehicles will be used before lower priority vehicles. The vehicle priorities are a hint to the solver, but the main goal is to minimize the number of vehicles. The size of this array must be equal to fleet_size.</p></details>
<br>

NVIDIA cuOpt also supports secondary matrix initialization with <b>set_secondary_matrix</b>. This provides an option to add a secondary matrix, for example time matrix, that can be used to check constraints satisfiability. <details><summary>Read more</summary> <p>For instance, a secondary matrix can be used to model the time between locations with time windows referring to it while the solver could optimize for distance.</p></details>

In [5]:
def data_model_initialization(df, nodes, n_vehicles, vehicle_priorities, vehicle_capacity):

    my_data_model = cuopt.DataModelView(nodes, n_vehicles)

    # Add cost matrix information
    distances = build_matrix(df)
    my_data_model.set_matrix(distances)
    # If you wish to directly use the x, y coordinates you can use the set_coordinates API as below:
    # my_data_model.set_coordinates(df['x'], df['y'])

    # Set vehicle priorities
    if len(vehicle_priorities) > 0:
        my_data_model.set_vehicle_priorities(vehicle_priorities)
        
    capacity = cudf.Series(vehicle_capacity)
    
    # Add capacity dimension
    my_data_model.add_capacity_dimension("demand", df['demand'], capacity)
    
    # Add delivery time windows
    my_data_model.add_time_window(df['earliest_time'], df['latest_time'], df['service_time'])

    return my_data_model

### **Step 3: Initialize and set up the Solver**
Once we have our data model, we need to initialize our solver with the data model and all the time, capacity and solver constraints.

<br>

* Adding capacity constraints:

A vehicle can have various capacity constraints like weight, volume and number of orders it can carry. To add these, we can use the <b>add_capacity_dimension</b> which takes in information regarding the demand value for each location and the capacity value for each vehicle in the fleet. NVIDIA cuOpt supports multiple capacity dimensions, however, we have restricted support to only single capacity dimension for this demonstration.

<br>

* Adding time constraints:

Each order demand has a time slot for delivery i.e., a time constraint that denotes what is the earliest time and latest time the order needs to be delivered. It also specifies the service time i.e., the amount of time spent at each location for the delivery. These constraints can be added using <b>add_time_windows</b> which takes in the earliest time, latest time, service time for each delivery. We can additionally provide the penalties of each location allowing to model node priority.
<details><summary>Read more</summary>
<p>
Consider a scenario at peak times where you might have a few orders to deliver but not enough vehicles. It might turn out that a solution that meets all delivery time window requirements is not feasible. In these cases, even though an optimal solution cannot be found, it might be a better strategy to allow a little deviation from our delivery time windows instead of leaving our customers without their orders. We can use <b>set_solution_scope</b> to soft time windows which will expand the route search to unfeasible regions. This feature is not available in the restricted version of NVIDIA cuOpt we are using for this demonstration.</p></details>

<br>

* Adding solver constraints:

Now consider an opposite scenario where we have more number of vehicles than required for delivery. By default, NVIDIA cuOpt minimizes the number of vehicles in its solution. However, if we have more vehicles available, why not use all of them? For this purpose, we can utilize the <b>set_min_vehicles</b> API to request a minimum number of vehicles in the solution although the resulting solution may not be optimal.

<b>set_number_of_climbers</b> can be used to set the number of climbers for the local search optimization. Number of climbers are number of instances trying to find solutions which start at different initial status. Hence, higher the number of climbers, higher the probability of getting better results in long run. <details><summary>Read more</summary>
<p>However, if it is desired to have good results in a short time, lower number of climbers is better. By default, the number of climbers is chosen by considering occupancy of a small GPU and experimented run-time vs number of climbers trade-off (i.e. the best result in shortest time).</p></details>

Solving time varies a lot with number of locations and default value may not be appropriate in some scenarios. So use <b>set_time_limit</b>, set a solving time and experiment with it, after a while cost of solution will converge. <details><summary>Read more</summary>
<p>Higher the solving time, higher the accuracy. This may impact the accuracy. By default it is set to num_locations/5 by considering number of climbers vs run-time trade-off.</p></details>
<br>

<details><summary><b>More options</b></summary>
<p>
By default NVIDIA cuOpt includes the return trip back to depot in all computed routes. In certain situations, for example if we use contactor vehicles instead of store vehicles, we don't intend for the vehicle to return to the depot. In this case we can use the <b>drop_return_trips</b> to control if individual vehicles in the fleet return to the depot after the last stop.

NVIDIA cuOpt also provides the flexibility of optimizing the routes for you if you have previously computed routes. This can be done by adding the initial solution via <b>add_guess</b>

We could also budget to let vehicles wait at locations up to a specified time with <b>set_slack_max</b>. By default vehicles can wait an infinite amount of time.</p></details>

In [6]:
def solver_initialization(df, my_data_model):
    my_solver = cuopt.Solver(my_data_model)

    
    # Set seconds update and climbers
    my_solver.set_time_limit(20)
    my_solver.set_number_of_climbers(2048)

    return my_solver

### **Step 4: Call solve and get the routes**
Now that you have fed NVIDIA cuOpt all your data and requirements, it's time to call for solve and let NVIDIA cuOpt find you your optimized routes!

Once NVIDIA cuOpt has found your solution, you can view the various details of the route as well as the route itself.
<b>get_cost</b> gives you the total cost of the final solution.
<b>get_vehicle_count</b> returns the number of vehicles used in the optimal solution.
And finally, to view routes you can use <b>get_routes</b> to get the route, vehicle ids for each stop and the arrival stamp in a Cuda Dataframe.

In [7]:
def call_solve(my_solver):
    routing_solution = my_solver.solve()
    final_cost = routing_solution.get_cost()
    vehicle_count = routing_solution.get_vehicle_count()
    cu_status = routing_solution.get_status()
    if cu_status != 0:
        print("""
        --------------------------------------------------------------------------------------------------
          !!!!!!!!!!!!        Failed: Solution within constraints could not be found     !!!!!!!!!!!!!!!!
        -------------------------------------------------------------------------------------------------- """)
    else:
        print("Final Cost         : ", final_cost) 
        print("Number of Vehicles : ", vehicle_count)
    return routing_solution

### **Bringing it together**
Let's combine these 4 simple steps to make a function that launches NVIDIA cuOpt on our initial data

In [8]:
# Run NVIDIA cuopt on given dataset, for number vehicles with particular capacity and vehicle_priorities.
def run_cuopt(df, n_vehicles, vehicle_capacity, vehicle_priorities=[]):

    nodes = df["demand"].shape[0]

    my_data_model = data_model_initialization(df, nodes, n_vehicles, vehicle_priorities, vehicle_capacity)

    my_solver =  solver_initialization(df, my_data_model)

    # Solve for routes and cost
    routing_solution = call_solve(my_solver)
    
    return routing_solution

Looks like we are all set to see NVIDIA cuOpt in action!

### **Solve on Gehring & Homeberger instance data**
We have our Homeberger instance data read into a Cuda Dataframe. It specifies number of vehicles as 250 and Capacity as 200. So, let's try to find a solution for it with 250 vehicles each with capacity 200.

In [9]:
vehicle_capacity = cudf.Series([vehicle_capacity]*n_vehicles)
solution = run_cuopt(df, n_vehicles, vehicle_capacity)
print("""
------------------------------------------------------------------------------------
                                     Routes
------------------------------------------------------------------------------------
""")
#solution.display_routes()

Final Cost         :  42946.4375
Number of Vehicles :  100

------------------------------------------------------------------------------------
                                     Routes
------------------------------------------------------------------------------------



NVIDIA cuOpt is able to provided a solution using 100 (best known solution) of the 250 available vehicles to service 1000 locations. This world class solution was provided by cuOpt in 20 seconds !

### **Unfeasible solution case**
Suppose we only have 70 vehicles each with capacity 200. 
We can observe that it is not possible to find a routing solution that satisfies the time and capacity constraints.

In [10]:
n_vehicles = 70
vehicle_capacity = 200
vehicle_capacity = [vehicle_capacity]*n_vehicles
solution = run_cuopt(df, n_vehicles, vehicle_capacity)


        --------------------------------------------------------------------------------------------------
          !!!!!!!!!!!!        Failed: Solution within constraints could not be found     !!!!!!!!!!!!!!!!
        -------------------------------------------------------------------------------------------------- 


### **Mixed Fleet and Priorities**
![Image of Fleet](https://www.universeoptics.com/wp-content/uploads/commercial_vehicle.jpg)

What if we have 130 more backup vehicles with capacity of 100 available to us.
However, these should be used only when we have no more store vehicles. 

This can be done by initializing mixed fleet and setting priorities.                                             
Store vehicles are set to have higher priorities (priority set to 0) than backup vehicles (priority set to 1)

In [11]:
store_vehicles = 70
store_vehicle_capacity = 200
backup_vehicles = 130
backup_vehicle_capacity = 100
n_vehicles = store_vehicles + backup_vehicles
vehicle_capacity = [store_vehicle_capacity]*store_vehicles + [backup_vehicle_capacity]*backup_vehicles
vehicle_priorities = cudf.Series([0 for i in range(0, 70)] + [100 for i in range(70, 200)])

# Run cuopt with mixed fleet and priorities
solution = run_cuopt(df, n_vehicles, vehicle_capacity, vehicle_priorities)

Final Cost         :  55915.90625
Number of Vehicles :  120


You can observe in the resulting routes that all 70 store vehicles are used first.

In [12]:

print("""
------------------------------------------------------------------------------------
                                     Routes
------------------------------------------------------------------------------------
""")
solution.display_routes()


------------------------------------------------------------------------------------
                                     Routes
------------------------------------------------------------------------------------

Vehicle-0 starts at: 65.92, completes at: 954.58, travel time: 888.66,  Route : 
0->72->573->337->613->947->588->439->751->0 

Vehicle-1 starts at: 0.0, completes at: 1463.78, travel time: 1463.78,  Route : 
0->599->755->704->99->10->649->556->239->863->330->826->0 

Vehicle-2 starts at: 0.0, completes at: 1112.93, travel time: 1112.93,  Route : 
0->856->758->774->478->410->5->477->398->548->0 

Vehicle-3 starts at: 0.0, completes at: 1072.45, travel time: 1072.45,  Route : 
0->997->87->585->365->120->521->996->21->583->0 

Vehicle-4 starts at: 0.0, completes at: 1779.62, travel time: 1779.62,  Route : 
0->95->834->557->381->271->215->630->80->236->264->750->642->310->0 

Vehicle-5 starts at: 0.0, completes at: 1023.21, travel time: 1023.21,  Route : 
0->615->667->808->926-


## **Let's Try to Visualize Our Routes**

Great! Now that we have found our optimized routes it's time to send the vehicles on their way.  

**Note**: Veroviz is used just to visualize the routes and it’s not part of NVIDIA cuOpt. And we are not comparing  NVIDIA cuOpt against veroviz as routing solution.

We have used **veroviz** to help us with the visualization.          
Suppose we have 6 vehicles ready to be loaded, let's pick 6 routes, one for each vehicle and try to visualize the path they will be taking.

In [13]:
# Get X and Y coordinates for nodes in 6 selected routes
routes_df = solution.get_route()

# Some times the creation of map might fail which depends on results, which inturn depends on GPU and other factors,
# so please change the routes chosen which will fix the problem. Current [1, 4) and [94, 97) have been chosen.

truck_ids = routes_df['truck_id'].unique()[1:4].to_arrow().to_pylist()+routes_df['truck_id'].unique()[94:97].to_arrow().to_pylist()
routes_df = routes_df[routes_df['truck_id'].isin(truck_ids)]
pdf = df.to_pandas()
routes_df = routes_df.to_pandas()
routes_df = routes_df.merge(pdf, how='left', left_on='route', right_on='vertex')
routes_df = routes_df[['vertex', 'x', 'y', 'truck_id']]

### Map generic X and Y coordinates to actual Longitude and Latitude
This will help in visualization on a map. We have scaled the coordinates to select **NVIDIA** headquarters as our depot!

In [14]:
nodes, routes = helper_map.map_XY_to_LongLat(routes_df, truck_ids)

/home/cuopt_user/miniconda/conda/envs/cuopt/lib/python3.8/site-packages/pyproj/crs/crs.py:130: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/home/cuopt_user/miniconda/conda/envs/cuopt/lib/python3.8/site-packages/pyproj/crs/crs.py:130: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


### Mark nodes and find paths using veroviz

In [15]:
%%capture text
leaflet = helper_map.get_vrv_leaflet(nodes, routes)

### Display nodes and routes on the map

In [16]:
leaflet



# CONGRATULATIONS !
You have completed the course.  For additional information on NVIDIA cuOpt consider joining the [NVIDIA cuOpt Early Access Program](https://developer.nvidia.com/cuopt-logistics-optimization/early-access-program)

In [17]:
import IPython
app = IPython.Application.instance()
app.kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}

<center><a href="https://www.nvidia.com/dli"> <img src="images/DLI_Header.png" alt="Header" style="width: 400px;"/> </a></center>